In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #for plotting
from sklearn.ensemble import RandomForestClassifier #for the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz #plot tree
from sklearn.metrics import roc_curve, auc #for model evaluation
from sklearn.metrics import classification_report #for model evaluation
from sklearn.metrics import confusion_matrix #for model evaluation
from sklearn.model_selection import train_test_split #for data splitting
np.random.seed(1) #ensure reproducibility

pd.options.mode.chained_assignment = None  #hide any pandas warnings

In [ ]:
# Data Cleaning

In [2]:
import pandas as pd
import numpy as np
import dame_flame
import random
import matplotlib.pyplot as plt

In [77]:
LIFT_reg = pd.read_csv("/Users/JennyH/cisil-causal-inference/Data/LIFT_registry.csv")

In [78]:
LIFT_reg.head()

,Age,RaceDesc,LanguageSpoken,CardIssuingAgency,FIPS,ExpirationDate,DateIssuedToCardHolder,Initial Load,Study Card,card_id
0,35,Black or African American,Other,KCMCCS,5.303303e+11,8/31/2019 0:00,2017-06-15 09:57:00,NaN,NaN,568-1
1,55,White,English,YWCA,5.303300e+11,7/31/2019 0:00,2017-06-05 14:04:00,NaN,NaN,2690-1
2,58,White,English,King County Public Health,5.303300e+11,11/30/2021 0:00,2019-08-28 13:47:00,NaN,NaN,2690-2
3,45,Asian,English,King County Public Health,5.303301e+11,5/31/2021 0:00,2019-06-30 17:47:00,NaN,NaN,2706-1
4,62,Asian,Chinese,Promo Account,5.303301e+11,3/31/2019 0:00,2017-03-31 16:25:00,NaN,NaN,2708-1


In [79]:
LIFT_reg.columns = [c.replace(' ', '_') for c in LIFT_reg.columns]

In [80]:
p1 = LIFT_reg[(LIFT_reg.Study_Card != "Subsidized Annual Pass") 
         & (LIFT_reg.Age >= 18) 
         & (LIFT_reg.DateIssuedToCardHolder >= "2019-03-01") 
         & (LIFT_reg.DateIssuedToCardHolder <= "2019-07-01") 
         & (LIFT_reg.CardIssuingAgency.isin(["CSO Auburn", "CSO Capitol Hill", "CSO White Center"]))]

In [81]:
p1 # 1368 rows

,Age,RaceDesc,LanguageSpoken,CardIssuingAgency,FIPS,ExpirationDate,DateIssuedToCardHolder,Initial_Load,Study_Card,card_id
158,50,American Indian or Alaska Native,English,CSO White Center,5.303301e+11,2/28/2022 0:00,2019-04-01 13:38:00,10,NaN,3228-1
364,66,White,English,CSO Capitol Hill,5.303301e+11,5/31/2021 0:00,2019-05-29 13:04:00,Passport,Phase1 Treatment,3970-2
1100,50,NaN,NaN,CSO Capitol Hill,5.303301e+11,2/28/2022 0:00,2019-04-26 13:55:00,10,NaN,6476-2
1232,64,White,English,CSO White Center,5.303303e+11,2/28/2022 0:00,2019-04-25 11:35:00,10,NaN,6950-1
2194,56,Black or African American,English,CSO Capitol Hill,5.303301e+11,10/31/2021 0:00,2019-05-13 11:43:00,10,NaN,10168-2
...,...,...,...,...,...,...,...,...,...,...
74865,23,Other,Other,CSO White Center,5.306105e+11,2/28/2022 0:00,2019-06-28 13:30:00,10,NaN,199252-1
74866,38,White,English,CSO White Center,5.303303e+11,2/28/2022 0:00,2019-06-28 13:35:00,10,NaN,199254-1
74868,65,Not Specified,English,CSO Capitol Hill,5.303301e+11,10/31/2021 0:00,2019-06-28 13:45:00,10,NaN,199258-1
74875,29,Not Specified,English,CSO Capitol Hill,NaN,10/31/2021 0:00,2019-06-28 14:23:00,10,NaN,199272-1


In [82]:
d = {"Passport":1, "10": 0}
p1['Initial_Load'] = p1['Initial_Load'].map(d)

In [83]:
# I bin all continuous variables.
p1['Age_bin']=pd.cut(x = p1['Age'],
                        bins = [17,25,35,45,55,65,100], 
                        labels = [0,1,2,3,4,5])

# I use one hot encoding for features with more than one category (ie smoking_status, work_type, age_bin)
prep_data = p1[['RaceDesc','LanguageSpoken', 'CardIssuingAgency']]
# prep_data
prep_data=pd.get_dummies(prep_data)
prep_data

p1.drop(columns=['RaceDesc','LanguageSpoken', 'CardIssuingAgency', "Study_Card"],inplace=True)

In [84]:
p1

,Age,FIPS,ExpirationDate,DateIssuedToCardHolder,Initial_Load,card_id,Age_bin
158,50,5.303301e+11,2/28/2022 0:00,2019-04-01 13:38:00,0,3228-1,3
364,66,5.303301e+11,5/31/2021 0:00,2019-05-29 13:04:00,1,3970-2,5
1100,50,5.303301e+11,2/28/2022 0:00,2019-04-26 13:55:00,0,6476-2,3
1232,64,5.303303e+11,2/28/2022 0:00,2019-04-25 11:35:00,0,6950-1,4
2194,56,5.303301e+11,10/31/2021 0:00,2019-05-13 11:43:00,0,10168-2,4
...,...,...,...,...,...,...,...
74865,23,5.306105e+11,2/28/2022 0:00,2019-06-28 13:30:00,0,199252-1,0
74866,38,5.303303e+11,2/28/2022 0:00,2019-06-28 13:35:00,0,199254-1,2
74868,65,5.303301e+11,10/31/2021 0:00,2019-06-28 13:45:00,0,199258-1,4
74875,29,NaN,10/31/2021 0:00,2019-06-28 14:23:00,0,199272-1,1


In [85]:
p1 = p1.merge(prep_data,left_index=True, right_index=True,how='left')
p1

,Age,FIPS,ExpirationDate,DateIssuedToCardHolder,Initial_Load,card_id,Age_bin,RaceDesc_American Indian or Alaska Native,RaceDesc_Asian,RaceDesc_Black or African American,...,LanguageSpoken_Other,LanguageSpoken_Russian,LanguageSpoken_Somali,LanguageSpoken_Spanish,LanguageSpoken_Tagalog,LanguageSpoken_Tigrinya,LanguageSpoken_Vietnamese,CardIssuingAgency_CSO Auburn,CardIssuingAgency_CSO Capitol Hill,CardIssuingAgency_CSO White Center
158,50,5.303301e+11,2/28/2022 0:00,2019-04-01 13:38:00,0,3228-1,3,1,0,0,...,0,0,0,0,0,0,0,0,0,1
364,66,5.303301e+11,5/31/2021 0:00,2019-05-29 13:04:00,1,3970-2,5,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1100,50,5.303301e+11,2/28/2022 0:00,2019-04-26 13:55:00,0,6476-2,3,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1232,64,5.303303e+11,2/28/2022 0:00,2019-04-25 11:35:00,0,6950-1,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2194,56,5.303301e+11,10/31/2021 0:00,2019-05-13 11:43:00,0,10168-2,4,0,0,1,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74865,23,5.306105e+11,2/28/2022 0:00,2019-06-28 13:30:00,0,199252-1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
74866,38,5.303303e+11,2/28/2022 0:00,2019-06-28 13:35:00,0,199254-1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,1
74868,65,5.303301e+11,10/31/2021 0:00,2019-06-28 13:45:00,0,199258-1,4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
74875,29,NaN,10/31/2021 0:00,2019-06-28 14:23:00,0,199272-1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [86]:
p1.describe()

,Age,FIPS,Initial_Load,RaceDesc_American Indian or Alaska Native,RaceDesc_Asian,RaceDesc_Black or African American,RaceDesc_Hispanic or Latino,RaceDesc_Multi-Racial,RaceDesc_Native Hawaiian or Other Pacific Islander,RaceDesc_Not Specified,...,LanguageSpoken_Other,LanguageSpoken_Russian,LanguageSpoken_Somali,LanguageSpoken_Spanish,LanguageSpoken_Tagalog,LanguageSpoken_Tigrinya,LanguageSpoken_Vietnamese,CardIssuingAgency_CSO Auburn,CardIssuingAgency_CSO Capitol Hill,CardIssuingAgency_CSO White Center
count,1368.000000,1.297000e+03,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,...,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.000000,1368.00000
mean,39.443713,5.303393e+11,0.135234,0.009503,0.046053,0.331140,0.070906,0.021199,0.024854,0.038012,...,0.030702,0.001462,0.002193,0.033626,0.002924,0.005117,0.001462,0.139620,0.301901,0.55848
std,13.543764,4.478080e+07,0.342098,0.097054,0.209676,0.470796,0.256762,0.144099,0.155736,0.191294,...,0.172572,0.038222,0.046795,0.180330,0.054014,0.071376,0.038222,0.346719,0.459250,0.49675
min,18.000000,5.303300e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,29.000000,5.303301e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,37.000000,5.303303e+11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000
75%,50.000000,5.303303e+11,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.00000
max,96.000000,5.306105e+11,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [87]:
# Deal with NULL values
# There are 201 null values for bmi. We will drop these values in the cell below.
p1.isnull().sum()

Age                                                    0
FIPS                                                  71
ExpirationDate                                         0
DateIssuedToCardHolder                                 0
Initial_Load                                           0
card_id                                                0
Age_bin                                                0
RaceDesc_American Indian or Alaska Native              0
RaceDesc_Asian                                         0
RaceDesc_Black or African American                     0
RaceDesc_Hispanic or Latino                            0
RaceDesc_Multi-Racial                                  0
RaceDesc_Native Hawaiian or Other Pacific Islander     0
RaceDesc_Not Specified                                 0
RaceDesc_Other                                         0
RaceDesc_White                                         0
LanguageSpoken_Chinese                                 0
LanguageSpoken_Dari            

In [88]:
#p1['Age_bin'].head()
#p1.iloc[[364], :]
p1[p1['Age_bin'].isna()]

,Age,FIPS,ExpirationDate,DateIssuedToCardHolder,Initial_Load,card_id,Age_bin,RaceDesc_American Indian or Alaska Native,RaceDesc_Asian,RaceDesc_Black or African American,...,LanguageSpoken_Other,LanguageSpoken_Russian,LanguageSpoken_Somali,LanguageSpoken_Spanish,LanguageSpoken_Tagalog,LanguageSpoken_Tigrinya,LanguageSpoken_Vietnamese,CardIssuingAgency_CSO Auburn,CardIssuingAgency_CSO Capitol Hill,CardIssuingAgency_CSO White Center


In [ ]:
# Combine the Response-variable (sales) with the features (registry).